In [ ]:
import matplotlib.pyplot as plt
import numpy
%matplotlib inline
import scipy.interpolate
import topo_builder

In [ ]:
# # Profile used at coasts
# x = numpy.linspace(-.25,1.25,100)
# x0 = 0; plt.axvline(x0, linestyle='--', color='k', label='x0')
# L = 1; plt.axvline(x0+L, linestyle='--', color='k', label='x0+L')
# lf = 0.125; plt.axvline(x0+lf*L, linestyle='--', color='b', label='x0+lf*L')
# bf = 0.125; plt.axvline(x0+(lf+bf)*L, linestyle='--', color='r', label='x0+(lf+bf)*L')
# sf = 0.5; plt.axvline(x0+(1-sf)*L, linestyle='--', color='g', label='x0+(1-sf)*L')
# sh = 0.05; plt.axhline(-sh, linestyle=':', label='sh')
# plt.plot(x, -topo_builder.topo.coastal_sprofile(x, x0, L, sh, lf, bf, sf) )
# plt.grid()
# plt.legend();

In [ ]:
# NeverWorld2 domain
NW2_lonW, NW2_lonE = 0, 60
NW2_latS, NW2_latN = -70, 70

D0 = 4000 # Nominal depth (m)
cd = 200 # Depth of coastal shelf (m)
drake = 2500 # Depth of Drake sill (m)
cw = 5 # Width of coastal shelf (degrees)

# Logical domain (grid points)
nj, ni = 140, 60

In [ ]:
def scurve(x, x0, dx):
    """Returns 0 for x<x0 or x>x+dx, and a cubic in between."""
    s = numpy.minimum(1, numpy.maximum(0, (x-x0)/dx))
    return (3 - 2*s)*( s*s )
def itau(ytau,taud,ys):
    """Returns a profile tau(ys) that uses s-curves between node,valuesa ytau,taud"""
    taus = 0.*ys
    ks = 0
    for i in range(len(ys)):
        y = ys[i]
        if y>=ytau[ks+1]:
            ks=min(len(ytau)-2,ks+1)
        taus[i] = taud[ks] + ( taud[ks+1] - taud[ks]) * scurve(y, ytau[ks], ytau[ks+1]-ytau[ks])
    return taus
ytau,taud = [-70,-45,-15,0,15,45,70], [0,.2,-0.1,-.02,-.1,.1,0]
ys = numpy.linspace(-70,70,100)
tau = itau(ytau, taud, ys)

In [ ]:
# New profile for Neverworld2
h2 = numpy.array([25, 50, 100, 125, 150, 175, 200, 225, 250, 300, 350, 400, 500, 550, 600])
print('Total depth =', h2.sum(), '# of layers =', h2.shape[0] )

print('h = ',end='')
for i in range(len(h2)): print(h2[i], end=', ')
print()
e2 = numpy.zeros( h2.shape[0]+1 )
e2[1:] = h2.cumsum()
ec2 = ( e2[:-1] + e2[1:] )/2
def rrr(e, d=1000, drho=5.5):
    fe = numpy.exp((4000-e)/d)
    fe = fe - fe[-1]
    fe = fe/fe[0]
    return 1028.1 - drho * fe
r2 = rrr(ec2).round(2)
print('Layer = ',end='')
for i in range(len(r2)): print(r2[i], end=', ')
print()

In [ ]:
# Side-b-side comparison for discussion purposes
fig,axs = plt.subplots(1,1,figsize=(10,5),dpi=300)

T = topo_builder.topo(nj, ni, dlon=NW2_lonE, dlat=NW2_latN-NW2_latS, lat0=NW2_latS, D=D0)
T.add_NS_coast(NW2_lonW, -40, 90, cw, cd)
T.add_NS_coast(NW2_lonE, -40, 90, cw, cd)
T.add_NS_coast(NW2_lonW, -90, -60, cw, cd)
T.add_NS_coast(NW2_lonE, -90, -60, cw, cd)
T.add_EW_coast(-360, 360, NW2_latS, cw, cd)
T.add_EW_coast(-360, 360, NW2_latN, cw, cd)
T.add_circular_ridge(NW2_lonW, -50, 10, 2, drake) # Drake sill
T.add_NS_ridge(30, -90, 90, 30, D0/2) # Mid-Atlantic ridge?

plt.subplot(1,3,1)
im = plt.pcolormesh(T.xg, T.yg, T.z, vmin=-4000);
plt.xlabel('Longitude [$^\circ$E]');
plt.ylabel('Latitude [$^\circ$N]');
plt.title('a) Topography');
cb = plt.colorbar(label='z [m]')

plt.subplot(1,3,2)
plt.plot(taud, ytau, 'x')
plt.plot(tau, ys)
plt.ylim(-70,70);
plt.ylabel('Latitude [$^\circ$N]');
plt.xlabel(r'$\tau^x$ (Pa)');
plt.grid();
plt.title('b) Wind stress');

plt.subplot(1,3,3)
plt.plot(r2, -ec2, 'o-')
plt.xlabel('Layer density [kg m$^{-3}$]')
plt.ylabel('z [m]')
plt.ylim(-4000,0)
plt.xlim(1022,1028.5)
plt.grid();
plt.title('c) Stratification');

plt.tight_layout()

# cb = fig.colorbar(im, ax=axs, orientation='vertical', fraction=.175)

plt.savefig('fig-1.png')